In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Data Pipeline & Workflow

This tutorial demonstrates how to:
1. **Regrid** data from standard Lat-Lon grids to the Cubed-Sphere.
2. Run a simulation with **I/O Monitors** to save results to disk.
3. **Post-process** output NetCDF files.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereAdvectionSolver, AdvectionConfig
from cubed_sphere.utils.io import NetCDFMonitor

In [ ]:
# 1. Regridding Calculation (Conceptual)
# Simulate incoming weather data on a Lat-Lon grid
lat = np.linspace(-90, 90, 180)
lon = np.linspace(0, 360, 360)
LON, LAT = np.meshgrid(lon, lat)

data_latlon = np.sin(np.deg2rad(LAT)) * np.cos(np.deg2rad(LON))

# Use solver infrastructure to map this onto the sphere
# (In a real scenario, use scipy.interpolate.griddata to interpolate onto solver.geometry nodes)
print(f"Source Data Shape: {data_latlon.shape}")

In [ ]:
# 2. Simulation with I/O
config = AdvectionConfig(N=16, backend='numpy')
solver = CubedSphereAdvectionSolver(config)
state = solver.get_initial_condition("gaussian")

# Attach a Monitor
output_file = "tutorial_output.nc"
monitor = NetCDFMonitor(filename=output_file, interval=0.1)
monitor.initialize(solver, state)

dt = 0.05
t = 0.0
for i in range(10): # Run 10 steps
    state = solver.step(t, state, dt)
    t += dt
    monitor.update(t, state)
    
monitor.close()
print(f"Simulation finished. Data saved to {output_file}")

In [ ]:
# 3. Post-Processing
# Load the saved NetCDF file
ds = xr.open_dataset(output_file)
print(ds)

# Plot the last frame of Face 0
last_frame = ds['face_0'].isel(time=-1)
plt.figure(figsize=(8,6))
last_frame.plot()
plt.title("Post-processed Result (Face 0)")
plt.show()